In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import argparse
import time
import random
import math
import numpy as np
from runx.logx import logx
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
#from models import ResNet18
from classifier import CNN
from utils import load_dataset, init_func, Rand_Augment
from deeplearning import train_target_model, test_target_model, train_shadow_model, test_shadow_model
from attack import AdversaryOne_Feature, AdversaryOne_evaluation, AdversaryTwo_HopSkipJump, AdversaryTwo_QEBA, AdversaryTwo_SaltandPepperNoise
from cert_radius.certify import certify


action = -1

import torch.nn as nn

# Define Net class (Define Network)

N_ACTIONS = 2               
N_STATES = 20

BATCH_SIZE = 32                                 # Number of samples
LR = 0.01                                       # Learning rate
EPSILON = 0.99                                   # Greedy policy
GAMMA = 0.9                                     # Reward discount
TARGET_REPLACE_ITER = 100                       # Target network update frequency
MEMORY_CAPACITY = 2000                          # Memory capacity

class Net(nn.Module):
    def __init__(self):                                                         # Define a series of attributes for Net
        super(Net, self).__init__()                                             # Equivalent to nn.Module.__init__()

        self.fc1 = nn.Linear(N_STATES, 50)                                      # Set the first fully connected layer (input layer to hidden layer): from state neurons to 50 neurons
        self.fc1.weight.data.normal_(0, 0.1)                                    # Weight initialization (normal distribution with mean 0, variance 0.1)
        self.out = nn.Linear(50, N_ACTIONS)                                     # Set the second fully connected layer (hidden layer to output layer): from 50 neurons to action neurons
        self.out.weight.data.normal_(0, 0.1)                                    # Weight initialization (normal distribution with mean 0, variance 0.1)

    def forward(self, x):                                                       # Define forward function (x is state)
        x = F.relu(self.fc1(x))                                                 # Connect input layer to hidden layer, and use ReLU activation function to process values from hidden layer
        actions_value = self.out(x)                                             # Connect hidden layer to output layer, get the final output value (i.e., action value)
        return actions_value 
    
class DQN(object):
    def __init__(self):                                                         # Define a series of attributes for DQN
        self.eval_net, self.target_net = Net(), Net()                           # Use Net to create two neural networks: evaluation network and target network
        self.learn_step_counter = 0                                             # For target updating
        self.memory_counter = 0                                                 # For storing memory
        self.memory = np.zeros((MEMORY_CAPACITY, N_STATES * 2 + 2))             # Initialize memory, one row represents one transition
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=LR)    # Use Adam optimizer (input is the parameters of evaluation network and learning rate)
        self.loss_func = nn.MSELoss()                                           # Use Mean Squared Error loss function (loss(xi, yi)=(xi-yi)^2)

    def choose_action(self, x):                                                 # Define action selection function (x is state)
        x = torch.unsqueeze(torch.FloatTensor(x), 0)                            # Convert x to 32-bit floating point form, and add a dimension of size 1 at dim=0
        if np.random.uniform() < EPSILON:                                       # Generate a random number in [0, 1), if less than EPSILON, choose the optimal action
            actions_value = self.eval_net.forward(x)                            # Get action values by forward propagation of evaluation network with input state x
            action = torch.max(actions_value, 1)[1].data.numpy()                # Output the index of the maximum value of each row, and convert to numpy ndarray form
            action = action[0]                                                  # Output the first number of action
        else:                                                                   # Randomly choose action
            action = np.random.randint(0, N_ACTIONS)                            # Here action is randomly equal to 0 or 1 (N_ACTIONS = 2)
        return action                                                           # Return the selected action (0 or 1)

    def store_transition(self, s, a, r, s_):                                    # Define memory storage function (here input is a transition)
        transition = np.hstack((s, [a, r], s_))                                 # Concatenate arrays horizontally
        index = self.memory_counter % MEMORY_CAPACITY                           # Get the row number where transition will be placed
        self.memory[index, :] = transition                                      # Place the transition
        self.memory_counter += 1                                                # Increment memory_counter by 1

    def learn(self):                                                            # Define learning function (start learning after memory is full)
        if self.learn_step_counter % TARGET_REPLACE_ITER == 0:                  # Trigger at the beginning, then trigger every 100 steps
            self.target_net.load_state_dict(self.eval_net.state_dict())         # Assign parameters of evaluation network to target network
        self.learn_step_counter += 1                                            # Increment learn_step_counter by 1

        sample_index = np.random.choice(MEMORY_CAPACITY, BATCH_SIZE)            # Randomly select 32 numbers in [0, 2000), may repeat
        b_memory = self.memory[sample_index, :]                                 # Extract 32 transitions corresponding to 32 indexes, store in b_memory
        b_s = torch.FloatTensor(b_memory[:, :N_STATES])
        b_a = torch.LongTensor(b_memory[:, N_STATES:N_STATES+1].astype(int))
        b_r = torch.FloatTensor(b_memory[:, N_STATES+1:N_STATES+2])
        b_s_ = torch.FloatTensor(b_memory[:, -N_STATES:])

        q_eval = self.eval_net(b_s).gather(1, b_a)
        q_next = self.target_net(b_s_).detach()
        q_target = b_r + GAMMA * q_next.max(1)[0].view(BATCH_SIZE, 1)
        loss = self.loss_func(q_eval, q_target)
        self.optimizer.zero_grad()                                      # Clear the residual updated parameters from the previous step
        loss.backward()                                                 # Backpropagation of error, calculate parameter update value
        self.optimizer.step()
        
def Train_Target_Model(args):
    split_size = args.Split_Size[args.dataset_ID]
    dataset = args.datasets[args.dataset_ID]
    dqn = DQN()
    state_list = []
    action_list = []
    reward_list = []
    for idx, cluster in enumerate(split_size):
        torch.cuda.empty_cache() 
        logx.initialize(logdir=args.logdir + '/target/' + str(cluster), coolname=False, tensorboard=False)
        train_loader, test_loader = load_dataset(args, dataset, cluster, mode=args.mode_type)
        targetmodel = CNN('CNN7', dataset)
        targetmodel.apply(init_func)
        targetmodel = nn.DataParallel(targetmodel.cuda())
        optimizer = optim.Adam(targetmodel.parameters(), lr=args.lr)
        logx.msg('======================Train_Target_Model {} ===================='.format(cluster))
        for epoch in range(1, args.epochs + 1):
            train_target_model(args, targetmodel, train_loader, optimizer, epoch)
            test_target_model(args, targetmodel, test_loader, epoch, save=True)


def Train_Shadow_Model(args):
    split_size = args.Split_Size[args.dataset_ID]
    dataset = args.datasets[args.dataset_ID]
    save = True
    for idx, cluster in enumerate(split_size):
        torch.cuda.empty_cache()
        train_loader = load_dataset(args, dataset, cluster, mode=args.mode_type)
        targetmodel = CNN('CNN7', dataset)
        shadowmodel = CNN('CNN7', dataset)
        
        targetmodel.apply(init_func)
        shadowmodel.apply(init_func)
        targetmodel = nn.DataParallel(targetmodel.cuda())
        shadowmodel = nn.DataParallel(shadowmodel.cuda())
        
        state_dict, _ =  logx.load_model(path=args.logdir + '/target/' + str(cluster) + '/best_checkpoint_ep.pth')
        targetmodel.load_state_dict(state_dict)
        dqn = DQN()
        state_list = []
        action_list = []
        reward_list = []
        
        for i in range(20):
            state_list.append(1)
            
        logx.initialize(logdir=args.logdir + '/shadow/'+ str(cluster), coolname=False, tensorboard=False)
        optimizer = optim.Adam(shadowmodel.parameters(), lr=args.lr)
        logx.msg('======================Train_Shadow_Model {} ===================='.format(cluster))
        
        j = 0
        accuracy1 = []
        num_pred = 0
        for epoch in range(1, args.epochs + 1):
            
            if j <= 20:
                action_list.append(1)
            else:
                action_list.append(dqn.choose_action(state_list[j]))

            targetmodel.eval()
            shadowmodel.train()
                               
            for batch_idx, (data, _) in enumerate(train_loader):
                data = data.cuda()
                output = targetmodel(data)
                _, target = output.max(1)
                optimizer.zero_grad()
                output = shadowmodel(data)
                loss = F.cross_entropy(output, target)
                loss.backward()
                optimizer.step()
                if batch_idx % args.log_interval == 0:
                    logx.msg('ShadowModel Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100. * batch_idx / len(train_loader),
                        loss.item()))
            
            targetmodel.eval()
            shadowmodel.eval()
            test_loss = 0
            correct = 0
            with torch.no_grad():
                for batch_idx, (data, _) in enumerate(train_loader):
                    data = data.cuda()
                    output = targetmodel(data)
                    _, target = output.max(1)

                    output = shadowmodel(data)
                    test_loss += F.cross_entropy(output, target).item()
                    pred = output.max(1, keepdim=True)[1]
                     
                    reference_tensor = pred
                    
                    if action_list[j] == 1:
                        random_array = np.random.randint(0, 10, size=(128, 1))
                        pred = torch.from_numpy(random_array).cuda()
                        pred = pred[:reference_tensor.shape[0]]
                        for k in range(len(target)):
                            target = torch.cat([target[:k], target[k+1:]], dim=0)
                            pred = torch.cat([pred[:k], pred[k+1:]], dim=0)
                    correct += pred.eq(target.view_as(pred)).sum().item()
                    accuracy1.append(correct / len(pred))
                    num_pred += len(pred)
            test_loss /= len(train_loader.dataset)
            accuracy = sum(accuracy1) / len(accuracy1)
         
            logx.msg('\nShadowModel Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                test_loss, correct, num_pred, accuracy))

            if save:
                save_dict = {
                    'epoch': epoch + 1,
                    'state_dict': shadowmodel.state_dict(),
                    'accuracy': accuracy}
                logx.save_model(
                    save_dict,
                    metric=accuracy,
                    epoch='',
                    higher_better=True)
            
            if action_list[j] == 1:
                state_list.append(1)
                reward_list.append(5)
            else:
                state_list.append(0)
                reward_list.append(-5)
            
            dqn.store_transition(state_list[0:20], action_list[j], reward_list[j], state_list[1:21])
            dqn.learn()
            state_list.pop(0)
            j += 1

def Train_Shadow_Model_ChangeDataSize(args):
    dataset = 'CIFAR100'
    split_size = [42000, 35000, 30000, 20000, 15000, 10000, 7000, 6000, 5000] 
    Nets = ['CNN3', 'CNN4', 'CNN5', 'CNN6', 'CNN7', 'CNN8', 'CNN9', 'CNN10', 'CNN11'] 
    targetmodel = CNN('CNN7', dataset)
    targetmodel = nn.DataParallel(targetmodel.cuda())
    state_dict, _ =  logx.load_model(path=args.logdir + '/target/7000/best_checkpoint_ep.pth')
    targetmodel.load_state_dict(state_dict)
    for net in Nets:
        for _, cluster in enumerate(split_size):
            torch.cuda.empty_cache()
            train_loader = load_dataset(args, dataset, cluster, mode='ChangeDataSize')
            shadowmodel = CNN(net, dataset)
            shadowmodel.apply(init_func)
            shadowmodel = nn.DataParallel(shadowmodel.cuda())
            logx.initialize(logdir=args.logdir + '/ChangeDataSize/' + net + '/' + str(cluster), coolname=False, tensorboard=False)
            optimizer = optim.Adam(shadowmodel.parameters(), lr=args.lr)
            logx.msg('======================Train_Shadow_Model_ChangeDataSize Size: {}  Nets: {}===================='.format(cluster, net))
            for epoch in range(1, args.epochs + 1):
                train_shadow_model(args, targetmodel, shadowmodel, train_loader, optimizer, epoch)
                test_shadow_model(args, targetmodel, shadowmodel, train_loader, epoch, save=True)

def AdversaryOne(args): ## loss or entropy or maximum
    logx.initialize(logdir=args.logdir + '/adversaryOne', coolname=False, tensorboard=False)
    split_size = args.Split_Size[args.dataset_ID]
    dataset = args.datasets[args.dataset_ID]
    AUC_Loss, AUC_Entropy, AUC_Maximum = [], [], []
    Distribution_Loss = []

    for cluster in split_size:
        torch.cuda.empty_cache()
        args.batch_size = 1
        data_loader = load_dataset(args, dataset, cluster, mode='adversary', max_num=2000)
        
        targetmodel = CNN('CNN7', dataset)
        targetmodel.apply(init_func)
        targetmodel = nn.DataParallel(targetmodel.cuda())
        shadowmodel = CNN('CNN7', dataset)
        shadowmodel.apply(init_func)
        shadowmodel = nn.DataParallel(shadowmodel.cuda())

        state_dict, _ =  logx.load_model(path=args.logdir + '/target/' + str(cluster) + '/best_checkpoint_ep.pth')
        targetmodel.load_state_dict(state_dict)
        targetmodel.eval()
        state_dict, _ =  logx.load_model(path=args.logdir + '/shadow/' + str(cluster) + '/best_checkpoint_ep.pth')
        shadowmodel.load_state_dict(state_dict)
        shadowmodel.eval()

        if args.advOne_metric == 'AUC':
            logx.msg('======================AdversaryOne AUC of Loss, Entropy, Maximum respectively cluster:{} ==================='.format(cluster))
            AUC_Loss, AUC_Entropy, AUC_Maximum = AdversaryOne_evaluation(args, targetmodel, shadowmodel, data_loader, cluster, AUC_Loss, AUC_Entropy, AUC_Maximum)
        elif args.advOne_metric == 'Loss_visual':
            Distribution_Loss = AdversaryOne_Feature(args, shadowmodel, data_loader, cluster, Distribution_Loss)
        print('finished')


def AdversaryOne_ChangeDataSize(args):
    split_size = [42000, 35000, 30000, 20000, 15000, 10000, 7000, 6000, 5000]
    dataset = 'CIFAR100'
    Nets = ['CNN3', 'CNN4', 'CNN5', 'CNN6', 'CNN7', 'CNN8', 'CNN9', 'CNN10', 'CNN11']
    data_loader = load_dataset(args, dataset, 7000, mode='adversary', max_num=2000)
    targetmodel = CNN('CNN7', dataset)
    targetmodel = nn.DataParallel(targetmodel.cuda())
    state_dict, _ =  logx.load_model(path=args.logdir + '/target/7000/best_checkpoint_ep.pth')
    targetmodel.load_state_dict(state_dict)
    targetmodel.eval()
    for net in Nets:
        AUC_Loss, AUC_Entropy, AUC_Maximum  = [], [], []
        for _, cluster in enumerate(split_size):
            torch.cuda.empty_cache()
            shadowmodel = CNN(net, dataset)
            shadowmodel = nn.DataParallel(shadowmodel.cuda())
            state_dict, _ =  logx.load_model(path=args.logdir + '/ChangeDataSize/' + net + '/' + str(cluster) + '/best_checkpoint_ep.pth')
            shadowmodel.load_state_dict(state_dict)
            shadowmodel.eval()
            AUC_Loss, AUC_Entropy, AUC_Maximum = AdversaryOne_evaluation(args, targetmodel, shadowmodel, data_loader, cluster, AUC_Loss, AUC_Entropy, AUC_Maximum)
        df = pd.DataFrame()
        AUC_Loss = df.append(AUC_Loss, ignore_index=True)
        AUC_Loss.to_csv(args.logdir + '/ChangeDataSize/' + net + '/AUC_Loss.csv')



def AdversaryTwo(args, Random_Data=False):
    if Random_Data:
        args.Split_Size = [[100], [2000], [100], [100]]
        img_sizes = [(3,32,32), (3,32,32), (3,64,64), (3, 128, 128)] 
    split_size = args.Split_Size[args.dataset_ID]
    dataset = args.datasets[args.dataset_ID]
    num_class = args.num_classes[args.dataset_ID]
    
    logx.initialize(logdir=args.logdir + '/adversaryTwo', coolname=False, tensorboard=False)
    if args.blackadvattack == 'HopSkipJump':
        ITER = [50]
    elif args.blackadvattack == 'QEBA':
        ITER = [150]
    elif args.blackadvattack == 'SaltandPepperNoise':
        ITER = [-1]
    for maxitr in ITER:
        AUC_Dist, Distance = [], []
        for cluster in split_size:
            torch.cuda.empty_cache()
            args.batch_size = 1
            if Random_Data:
                fake_set = datasets.FakeData(size=10000, image_size=img_sizes[args.dataset_ID], num_classes=num_class, transform= transforms.Compose([Rand_Augment(), transforms.ToTensor()]))
                data_loader = DataLoader(fake_set, batch_size=args.batch_size, shuffle=False)
            else:
                data_loader = load_dataset(args, dataset, cluster, mode='adversary', max_num=200)
            targetmodel = CNN('CNN7', dataset)
            targetmodel = nn.DataParallel(targetmodel.cuda())
            
            state_dict, _ =  logx.load_model(path=args.logdir + '/target/' + str(cluster) + '/best_checkpoint_ep.pth')
            targetmodel.load_state_dict(state_dict)
            targetmodel.eval()
            
            if args.blackadvattack == 'HopSkipJump':
                AUC_Dist, Distance = AdversaryTwo_HopSkipJump(args, targetmodel, data_loader, cluster, AUC_Dist, Distance, Random_Data, maxitr)
            elif args.blackadvattack == 'QEBA':
                AUC_Dist, Distance = AdversaryTwo_QEBA(args, targetmodel, data_loader, cluster, AUC_Dist, Distance, Random_Data, maxitr)
            elif args.blackadvattack == 'SaltandPepperNoise':
                AUC_Dist, Distance = AdversaryTwo_SaltandPepperNoise(args, targetmodel, data_loader, cluster, AUC_Dist, Distance, Random_Data)

        df = pd.DataFrame()
        AUC_Dist = df.append(AUC_Dist, ignore_index=True)
        Distance = df.append(Distance, ignore_index=True)
        
        if Random_Data:
            AUC_Dist.to_csv(args.logdir + '/adversaryTwo/AUC_Dist_'+args.blackadvattack+'.csv')
            Distance.to_csv(args.logdir + '/adversaryTwo/Distance_Random_'+args.blackadvattack+'.csv')
        else:
            AUC_Dist.to_csv(args.logdir + '/adversaryTwo/AUC_Dist_'+args.blackadvattack + '.csv')
            Distance.to_csv(args.logdir + '/adversaryTwo/Distance_'+args.blackadvattack+'.csv')
        

def Decision_Radius(args):
    num_class = args.num_classes[args.dataset_ID]
    dataset = args.datasets[args.dataset_ID]
    split_size = args.Split_Size[args.dataset_ID]
    
    for _, cluster in enumerate(split_size):
        torch.cuda.empty_cache()
        mem_set, non_set, transform = load_dataset(args, dataset, cluster, mode='radius')
        targetmodel = CNN('CNN7', dataset)

        targetmodel = nn.DataParallel(targetmodel.cuda())
        state_dict, _ =  logx.load_model(path=args.logdir + '/target/' + str(cluster) + '/best_checkpoint_ep.pth')
        targetmodel.load_state_dict(state_dict)
        targetmodel.eval()

        logx.initialize(logdir=args.logdir + '/radius/' + str(cluster), coolname=False, tensorboard=False)

        max_num = 200 if 200 < len(mem_set) else len(mem_set)
        logx.msg('======================Starting Decision Radius Training Dataset ====================')
        certify(targetmodel, 'cuda', mem_set, transform, num_class,
                    mode='both', start_img=0, num_img=max_num, 
                    sigma=0.25, beta=16)

        logx.msg('======================Starting Decision Radius Testing Dataset ====================')
        certify(targetmodel, 'cuda', non_set, transform, num_class,
                mode='both', start_img=0, num_img=max_num, 
                sigma=0.25, beta=16)


##############################
def main(): 
    # Training settings
    parser = argparse.ArgumentParser(description='PyTorch Decision-based Membership Inference Attack Toy Example') 
    parser.add_argument('--train', default=True, type=bool,
                        help='train or attack')
    parser.add_argument('--dataset_ID', default=2, type=int, 
                        help='CIFAR10=0, CIFAR100=1, GTSRB=2, Face=3')
    parser.add_argument('--datasets', nargs='+',
                        default=['CIFAR10', 'CIFAR100', 'GTSRB', 'Face'])
    parser.add_argument('--num_classes', nargs='+',
                        default=[10, 100, 43, 19])
    parser.add_argument('--Split-Size', nargs='+',
                        default=[[3000, 2000, 1500, 1000, 500, 100],                     
                                [7000, 6000, 5000, 4000, 3000, 2000],  
                                [600, 500, 400, 300, 200, 100],             
                                [350, 300, 250, 200, 150, 100]]) 
    parser.add_argument('--batch-size', nargs='+', default=128, metavar='N',
                        help='input batch size for training (default: 64)')
    parser.add_argument('--epochs', type=int, default=200, metavar='N',
                        help='number of epochs to train (default: 200)')
    parser.add_argument('--lr', type=float, default=0.001, metavar='LR',
                        help='learning rate (default: 0.001 for adam; 0.1 for SGD)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                        help='SGD momentum (default: 0.5)')
    parser.add_argument('--cuda', default=True, type=bool,
                        help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--blackadvattack', default='HopSkipJump', type=str,
                        help='adversaryTwo uses the adv attack the target Model: HopSkipJump; QEBA')
    parser.add_argument('--logdir', type=str, default='',
                        help='target log directory')
    parser.add_argument('--mode_type', type=str, default='',
                        help='the type of action referring to the load dataset')
    parser.add_argument('--advOne_metric', type=str, default='AUC', help='AUC of Loss, Entropy, Maximum respectively; or Loss_visual')
    
    args = parser.parse_args(args=[])

    for dataset_idx in [0,1]:
        args.dataset_ID = dataset_idx
        args.logdir = 'results' + '/' + args.datasets[args.dataset_ID]
        action = 3
        if action == 0:
            args.mode_type = 'target'
            Train_Target_Model(args)
        elif action == 1:
            args.mode_type = 'shadow'
            Train_Shadow_Model(args)
        elif action == 2: 
            args.logdir = 'results/CIFAR100' 
            Train_Shadow_Model_ChangeDataSize(args)
        elif action == 3:
            AdversaryOne(args)
        elif action == 4:    
            args.logdir = 'results/CIFAR100'  
            AdversaryOne_ChangeDataSize(args)
        elif action == 5:
            AdversaryTwo(args, Random_Data=False)
        elif action == 6:
            Decision_Radius(args)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'runx'